In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

In [9]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [10]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):
/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.

In [11]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):
/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.

In [14]:
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        model = keras.Sequential()
        model.add(layers.Flatten())
        model.add(
            layers.Dense(
                units=hp.Int("units", min_value=32, max_value=512, step=32),
                activation="relu",
            )
        )
        model.add(layers.Dense(10, activation="softmax"))
        model.compile(
            optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"],
        )
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            # Tune whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )


In [ ]:
    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)

In [ ]:
class ModeloRedes:
    def __init__(self, dataset, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = None
        self.y_test = None
        self.x_train = None
        self.y_train = None
        self.dataset_trabajo = dataset[:cantidad_filas]
        self.obtener_datasets()
        self.entrenar_modelo()

    def obtener_datasets(self):    
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.dataset_trabajo.loc[:,self.variables_predictoras],self.dataset_trabajo.loc[:,'property_price'],test_size=0.2)
        self.escalar_datasets()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        precios_pred=self.modelo.predict(df_para_predecir)
        mse=mean_squared_error(df_precios,precios_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n',
        self.dataset_trabajo.shape ,'\n')

In [15]:
hp = keras_tuner.HyperParameters()
hypermodel = MyHyperModel()
model = hypermodel.build(hp)
hypermodel.fit(hp, model, np.random.rand(100, 28, 28), np.random.rand(100, 10))


NameError: name 'layers' is not defined